# Sensitivity calculations with the double spike toolbox

In [1]:
import doublespike as ds

It can be useful to know how the results of a double spike inversion change with small changes in the input parameters. This is called a sensitivity analysis, and requires the calculation of the derivatives of the model parameters with respect to the input parameters. The calculation of these derivatives is described in Appendix B of the manuscript. As an example, we will consider the Mo system studied by Burkhardt (2012, PhD thesis), which has the parameters:       

In [2]:
isodata_mo = ds.IsoData("Mo")
isodata_mo.standard = [
    0.147339,
    0.092129,
    0.158922,
    0.166717,
    0.095685,
    0.242256,
    0.096952,
]
isodata_mo.spike = [
    0.009058,
    0.005014,
    0.009198,
    0.012766,
    0.363510,
    0.028056,
    0.572398,
]
isodata_mo.isoinv = [95, 97, 98, 100]
prop = 0.4766  # proportion of spike in spike-sample mix

To describe the sensitivity we will work with isotopic ratios, with $^{95}$Mo as the denominator isotope. The code below calculates the isotopic ratios of interest:

In [3]:
di = 95  # denominator isotope, 95Mo
y, P, n, T, m, *_ = ds.ratiodata(isodata_mo, di, prop, alpha=0.0, beta=0.0)
invrat = isodata_mo.invrat()
rationame = isodata_mo.rationame(di)
print("standard:", n)
print("spike:", T)
print("measured:", m)
print("names:", rationame)
print("inversion ratios:", rationame[invrat])

standard: [0.92711519 0.57971206 1.04904922 0.60208782 1.52437045 0.61006028]
spike: [ 0.9847793   0.5451185   1.38791042 39.52054795  3.05022831 62.23070233]
measured: [0.93000208 0.57798017 1.06601391 2.55049523 1.60076074 3.69502601]
names: ['92Mo/95Mo' '94Mo/95Mo' '96Mo/95Mo' '97Mo/95Mo' '98Mo/95Mo' '100Mo/95Mo']
inversion ratios: ['97Mo/95Mo' '98Mo/95Mo' '100Mo/95Mo']


$\mathbf{y} = (\lambda, \alpha, \beta)$ is the vector of model parameters, $\mathbf{P}$ is the natural log of the ratio of isotope masses, $\mathbf{n}$ is the standard/unspiked run, $\mathbf{T}$ is the double spike, and $\mathbf{m}$ is the measurement. `invrat` gives the indices of the isotope ratios used the inversion.

Suppose we are interested in how the model parameters $\mathbf{y}$ change with small changes in the standard value $\mathbf{n}$, i.e. we want the derivative $\partial \mathbf{y} / \partial \mathbf{n}$ given in (27) of the manuscript. This matrix can be obtained as:

In [4]:
dydn, *_ = ds.sensitivity(y, P, n, T, m, invrat)
print(dydn)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.12008424e-02
   3.96659826e-03  1.05075944e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.62029455e+01
   2.39442129e+01 -1.08263908e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.59193754e+01
   2.46737147e+00 -1.01154810e+01]]


The middle row of this matrix is the derivative $\partial \alpha / \partial \mathbf{n}$, which tells us how the fractionation factor $\alpha$ changes with small changes in the standard values.

In [5]:
dalphadn = dydn[1, :]
print(dalphadn)
print(rationame)

[  0.           0.           0.          16.20294547  23.94421293
 -10.82639081]
['92Mo/95Mo' '94Mo/95Mo' '96Mo/95Mo' '97Mo/95Mo' '98Mo/95Mo' '100Mo/95Mo']


## Correction formulae

Sensitivity analysis can be used to produce formulae to correct for departures from mass dependent fractionation. Here we provide an example of this, reproducing a correction formula given in (36) of [Hu and Dauphas (2017, J. Anal. At. Spectrom.)](https://doi.org/10.1039/C7JA00187H) This formula provides a correction $\Delta$ to the 98Mo/95Mo ratio in $\delta$ units when given nucleosynthetic anomalies on $\mathbf{n}$ in $\epsilon$ units. The correction is of the form  
$$\Delta_i =\sum_j  S_{ij} \epsilon_j$$
for coefficients $S_{ij}$ obtained from the sensitivity analysis. Variations in $\mathbf{n}$ can be written in $\epsilon$ units as $d \epsilon_j = 10^4 d n_j / n_j$. Variations in $\delta$ units relate to the fractionation factor $\alpha$ as $d \delta_i = 10^3 P_i d \alpha$. The coefficients of the Hu and Dauphas correction formula are thus given by 
$$S_{ij} = - \frac{P_i n_j}{10} \frac{\partial \alpha}{\partial n_j} $$
and calculated as:

In [6]:
ni = 98  # numerator isotope of interest
i = isodata_mo.ratioidx(ni, di)  # index corresponding to the 98/95 ratio
correction = -P[i] * n * dalphadn / 10.0
print(correction)
print(rationame)

[-0.         -0.         -0.         -0.03035603 -0.11357486  0.02055169]
['92Mo/95Mo' '94Mo/95Mo' '96Mo/95Mo' '97Mo/95Mo' '98Mo/95Mo' '100Mo/95Mo']


The above reproduces the Hu and Dauphas (2017) formula in (36)
$$\Delta^{98/95}\text{Mo} = -0.0304 \epsilon^{97/95}\text{Mo} -0.1136 \epsilon^{98/95}\text{Mo} + 0.0206\epsilon^{100/95}\text{Mo}$$